In [19]:
import json

import pandas as pd

In [20]:
def read_json(filename):
    with open(filename, encoding="UTF-8") as f:
        return list(json.load(f).items())
pd.DataFrame(read_json("input_data/train_set_ocr.json"))

,0,1
0,test_hashed\pit37_v1\9d6f61c6-52ad-4546-85b7-b...,"POLA Lalwe poda PODATNIK POLA , CIEMNE _ URZAD..."
1,test_hashed\pit37_v1\f14ea197-70bd-45ae-9a22-d...,PlTaxpl POUA 'WYPELNIA PODATNIK POLA CIEMNE WY...
2,test_hashed\pit37_v1\070a521a-7038-4551-bc6a-b...,MyPfenl Wypelnic DulyMi DaukowanlUiTERAM ClRnY...
3,test_hashed\pit37_v1\f4d15b30-6e96-4452-b614-a...,PlTaxpl POLA JASNE WYPEŁNIA PODATNIK _POLA CIE...
4,test_hashed\pit37_v1\64d085a8-39a0-4887-859e-8...,PITaxpl LA Jash€ WyPelNIA PODiTŃIK PCU CieMne ...
...,...,...
10879,./test_hashed/specification/8a1bb716-156c-4eca...,"MSDS 002z4 Ello ~arte-Vewaa @WEre Hkc hm2 ""7c0..."
10880,./test_hashed/specification/e42169d6-f1c4-416a...,"June 1992 40 03-540 Appearance: Colorless, cle..."
10881,./test_hashed/form/9807fe93-30d5-422b-8844-fd1...,PULL MAILFILE REQUEST TO: Bagby RJR-10301 July...
10882,./test_hashed/handwritten/e7eac2d5-364d-4f82-b...,~Tclecen 8/23/38 SAJ Co z & RH cam bt meaured ...


In [21]:
def get_filename_hash(filename, separator):
    return filename[filename.rindex(separator) + 1:] if separator in filename else filename
get_filename_hash("./test_hashed/advertisement/00133f89-017d-44f5-a549-e11cf5b5221a.tiff", "//")

'./test_hashed/advertisement/00133f89-017d-44f5-a549-e11cf5b5221a.tiff'

In [22]:
label_to_label_id =  pd.read_pickle('input_data/id2label_final.pkl')

In [23]:
filename_hash_to_label = {get_filename_hash(key, separator="/"): value for key, value in read_json("input_data/train_labels_final.json")}

## Add label and label_id

In [24]:
# ./test_hashed/.ipynb_checkpoints/18d1aa9b-c3cf-41e3-95db-bd668ada2000.tiff in input_data/train_set_ocr.json

train_dataset = pd.DataFrame(read_json("input_data/train_set_ocr.json"), columns="filename text".split()).rename({"index": "filename"}, axis=1)
train_dataset["filename"] = train_dataset["filename"].apply(get_filename_hash, args=["/"])

not_found = []
for filename in train_dataset["filename"]:
    if filename not in filename_hash_to_label:
        not_found.append(filename)
print(f"Not found: {len(not_found)}")
train_dataset = train_dataset[~train_dataset["filename"].isin(not_found)]

train_dataset["label"] = train_dataset["filename"].apply(lambda f: filename_hash_to_label[f]).astype("category")
train_dataset["label_id"] = train_dataset["label"].apply(lambda l: label_to_label_id[l]).astype("category")
train_dataset.info()

assert all(label in label_to_label_id for label in train_dataset["label"].to_list())

train_dataset.head()

Not found: 35
<class 'pandas.core.frame.DataFrame'>
Index: 10849 entries, 0 to 10883
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   filename  10849 non-null  object  
 1   text      10849 non-null  object  
 2   label     10849 non-null  category
 3   label_id  10849 non-null  category
dtypes: category(2), object(2)
memory usage: 276.9+ KB


,filename,text,label,label_id
0,test_hashed\pit37_v1\9d6f61c6-52ad-4546-85b7-b...,"POLA Lalwe poda PODATNIK POLA , CIEMNE _ URZAD...",pit37_v1,10
1,test_hashed\pit37_v1\f14ea197-70bd-45ae-9a22-d...,PlTaxpl POUA 'WYPELNIA PODATNIK POLA CIEMNE WY...,pit37_v1,10
2,test_hashed\pit37_v1\070a521a-7038-4551-bc6a-b...,MyPfenl Wypelnic DulyMi DaukowanlUiTERAM ClRnY...,pit37_v1,10
3,test_hashed\pit37_v1\f4d15b30-6e96-4452-b614-a...,PlTaxpl POLA JASNE WYPEŁNIA PODATNIK _POLA CIE...,pit37_v1,10
4,test_hashed\pit37_v1\64d085a8-39a0-4887-859e-8...,PITaxpl LA Jash€ WyPelNIA PODiTŃIK PCU CieMne ...,pit37_v1,10


In [25]:
train_dataset[["label", "label_id"]].value_counts()

label                                       label_id
handwritten                                 5           618
presentation                                12          592
scientific_report                           16          584
advertisement                               0           580
scientific_publication                      15          577
file_folder                                 3           566
form                                        4           566
resume                                      14          565
email                                       2           563
memo                                        8           558
budget                                      1           556
invoice                                     6           555
letter                                      7           551
questionnaire                               13          551
specification                               17          529
news_article                                9  

In [26]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10849 entries, 0 to 10883
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   filename  10849 non-null  object  
 1   text      10849 non-null  object  
 2   label     10849 non-null  category
 3   label_id  10849 non-null  category
dtypes: category(2), object(2)
memory usage: 276.9+ KB


## Labels number

In [34]:
print(len(train_dataset["label"].unique()))

21


## Language Split

In [35]:
import string

In [36]:
def tokenize(text: str):
    text = text.lower()

    punctuation = set(string.punctuation) - {"-"}
    for punctuation_mark in punctuation:
        text = text.replace(punctuation_mark, "")
    text = "".join(c for c in text if not c.isdigit())

    word_list = [word.strip().strip("-") for word in text.split()]
    word_list = [word for word in word_list if word]
    return " ".join(word_list)

train_dataset["text"] = train_dataset["text"].apply(tokenize)
train_dataset.tail()

,filename,text,label,label_id
10879,8a1bb716-156c-4eca-a8cd-f726a5cf7032.tiff,msds z ello arte-vewaa were hkc hm c izod rory...,specification,17
10880,e42169d6-f1c4-416a-a7eb-4fc90fd3c086.tiff,june appearance colorless clear mobile liquid ...,specification,17
10881,9807fe93-30d5-422b-8844-fd16529c8e7d.tiff,pull mailfile request to bagby rjr july for a ...,form,4
10882,e7eac2d5-364d-4f82-bb9d-354f11159a51.tiff,tclecen saj co z rh cam bt meaured ja approval...,handwritten,5
10883,143d2d22-378e-4595-a0e4-a4d02f60ec3a.tiff,oy stat ed mey fot postu firtt licd ncicc is i...,advertisement,0


In [37]:
print("Mean number of characters:", train_dataset["text"].apply(lambda s: len(s)).mean())
print("Mean number of words:", train_dataset["text"].apply(lambda s: len(s.split())).mean())

Mean number of characters: 1023.0436906627339
Mean number of words: 154.70946631025902


In [38]:
train_dataset.to_csv("train_input_data_pl_and_eng.csv", index=False)
first_english_index = train_dataset["filename"].to_list().index("a64b06e6-8f0a-4fd7-ba12-0779c5560d9a.tiff")
train_dataset_pl = train_dataset.iloc[:first_english_index]
train_dataset_eng = train_dataset.iloc[first_english_index:]

In [39]:
train_dataset_pl.to_csv("train_input_data_pl.csv", index=False)
train_dataset_eng.to_csv("train_input_data_eng.csv", index=False)

In [40]:
print("Polish language in training dataset", train_dataset_pl.shape[0] / train_dataset.shape[0])

Polish language in training dataset 0.16914001290441516
